Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо; задача с тем, возьмёт ли человек кредит).<br>
В частности использовать:<br>
GaussianNB<br>
LogisticRegression <br>
Linear + PolynomialFeatures<br>
RandomForestClassifier <br>
При обучении использовать make_pipeline<br>
GridSearchCV для поиска параметров <br>
Посчитать матрики (f1 + точность + AUC).

In [1]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np
import os


In [2]:
# get names without target columns
def get_features(df: pd.DataFrame, target: str) -> list:
    features = df.columns.tolist()
    features.remove(target)

    return features


Открываем обработанную ранее таблицу с кредитной информацией, оставляем для работы все столбцы, кроме 'client_id'. В качестве целевого столбца принимаем 'open_account_flg'. В таблице удалены пропуски, заменен символ ',' на '.' в дробных значениях, категориальные столбцы закодированы числами.

In [3]:
csv_path = 'processing_credit_train.csv'

credit_df = pd.read_csv(csv_path)
credit_df = credit_df.drop('client_id', axis=1)

credit_df


,age,credit_month,tariff_id,monthly_income,credit_count,overdue_credit_count,open_account_flg,score_shk_real,credit_sum_real,marital_status_int,...,СЕВЕРОКАВКАЗСКИЙ_real,СЕВЕРОЗАПАДНЫЙ_real,OTHER_real,ПРИВОЛЖСКИЙ_real,ЮЖНЫЙ_real,СИБИРСКИЙ_real,УРАЛЬСКИЙ_real,ДАЛЬНЕВОСТОЧНЫЙ_real,gender_income,region_income
0,0.150943,10,0.104167,0.058201,0.142857,0.0,0,0.525060,0.228841,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.0
1,0.264151,10,0.104167,0.036508,0.333333,0.0,0,0.476400,0.119292,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.150943,14,0.000000,0.032804,0.095238,0.0,0,0.456324,0.071260,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0
3,0.358491,10,0.625000,0.068783,0.047619,0.0,0,0.708524,0.200508,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6
4,0.471698,10,0.104167,0.058201,0.333333,0.0,0,0.349508,0.277957,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161326,0.169811,12,0.104167,0.037037,0.285714,0.0,0,0.548348,0.314964,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
161327,0.113208,6,0.625000,0.026455,0.047619,0.0,0,0.587308,0.075554,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161328,0.245283,10,0.333333,0.037037,0.047619,0.0,0,0.426275,0.125824,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
161329,0.660377,12,0.104167,0.027513,0.095238,0.0,0,0.494398,0.017504,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
credit_df.describe()


,age,credit_month,tariff_id,monthly_income,credit_count,overdue_credit_count,open_account_flg,score_shk_real,credit_sum_real,marital_status_int,...,СЕВЕРОКАВКАЗСКИЙ_real,СЕВЕРОЗАПАДНЫЙ_real,OTHER_real,ПРИВОЛЖСКИЙ_real,ЮЖНЫЙ_real,СИБИРСКИЙ_real,УРАЛЬСКИЙ_real,ДАЛЬНЕВОСТОЧНЫЙ_real,gender_income,region_income
count,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,...,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000
mean,0.359307,10.990746,0.334086,0.037477,0.100359,0.015354,0.164922,0.476354,0.117596,2.176928,...,0.028370,0.141975,0.041548,0.212910,0.109031,0.092034,0.095574,0.057088,0.479765,0.274986
std,0.196233,3.542895,0.245450,0.026801,0.084637,0.070500,0.371111,0.125009,0.082084,0.735688,...,0.166029,0.349026,0.199555,0.409366,0.311678,0.289075,0.294007,0.232011,0.418772,0.348703
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.207547,10.000000,0.104167,0.021164,0.047619,0.000000,0.000000,0.386009,0.061299,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.320755,10.000000,0.333333,0.031746,0.095238,0.000000,0.000000,0.468757,0.092975,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000
75%,0.471698,12.000000,0.625000,0.047619,0.142857,0.000000,0.000000,0.559614,0.147173,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000,0.600000
max,1.000000,36.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# create list of features and target  
target = 'open_account_flg'
features = get_features(credit_df, target)
print(features)


['age', 'credit_month', 'tariff_id', 'monthly_income', 'credit_count', 'overdue_credit_count', 'score_shk_real', 'credit_sum_real', 'marital_status_int', 'job_position_int', 'education_int', 'ЦЕНТРАЛЬНЫЙ_real', 'СЕВЕРОКАВКАЗСКИЙ_real', 'СЕВЕРОЗАПАДНЫЙ_real', 'OTHER_real', 'ПРИВОЛЖСКИЙ_real', 'ЮЖНЫЙ_real', 'СИБИРСКИЙ_real', 'УРАЛЬСКИЙ_real', 'ДАЛЬНЕВОСТОЧНЫЙ_real', 'gender_income', 'region_income']


In [15]:
# general DataFrame for save methods info
analyze_df = pd.DataFrame(columns=['Model', 'F1_score', 'Accuracy', 'AUC'])


In [12]:
def calculate_metrics(model_name: str, y_prob, y_pred, y_target):
    global analyze_df

    f1 = f1_score(y_target, y_pred)
    print('F1 score:', f1)

    score = accuracy_score(y_target, y_pred)
    print('Accuracy:', score)

    auc = roc_auc_score(y_target, y_prob)
    print('AUC:', auc)
    
    if analyze_df.empty:
        analyze_df = pd.DataFrame(dict(zip(analyze_df.columns, [[model_name], [f1], [score], [auc]])))
    else:
        analyze_df = analyze_df._append(dict(zip(analyze_df.columns, 
                                                 [model_name, f1, score, auc])), ignore_index=True)


Ипользуем наивный Байесовский анализ (GaussianNB) 

In [6]:
# create Pipeline for normalize dataset and classification
gaussian_pipe = Pipeline(steps=[('scaler', MinMaxScaler(feature_range=(0, 1))), 
                                ('classifier', GaussianNB())])

# normalize all dataset in features columns
credit_df_norm = credit_df.__deepcopy__()
credit_df_norm[features] = gaussian_pipe['scaler'].fit_transform(credit_df_norm[features])

# split dataset to train and test
credit_df_train_norm, credit_df_test_norm = train_test_split(credit_df_norm, test_size=0.1, random_state=0)

# fit Pipeline classifier
pipe = gaussian_pipe.fit(credit_df_train_norm[features], credit_df_train_norm[target])

# show normalized DataFrame
credit_df_norm


,age,credit_month,tariff_id,monthly_income,credit_count,overdue_credit_count,open_account_flg,score_shk_real,credit_sum_real,marital_status_int,...,СЕВЕРОКАВКАЗСКИЙ_real,СЕВЕРОЗАПАДНЫЙ_real,OTHER_real,ПРИВОЛЖСКИЙ_real,ЮЖНЫЙ_real,СИБИРСКИЙ_real,УРАЛЬСКИЙ_real,ДАЛЬНЕВОСТОЧНЫЙ_real,gender_income,region_income
0,0.150943,0.212121,0.104167,0.058201,0.142857,0.0,0,0.525060,0.228841,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.0
1,0.264151,0.212121,0.104167,0.036508,0.333333,0.0,0,0.476400,0.119292,0.75,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.150943,0.333333,0.000000,0.032804,0.095238,0.0,0,0.456324,0.071260,0.50,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0
3,0.358491,0.212121,0.625000,0.068783,0.047619,0.0,0,0.708524,0.200508,0.50,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6
4,0.471698,0.212121,0.104167,0.058201,0.333333,0.0,0,0.349508,0.277957,0.50,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161326,0.169811,0.272727,0.104167,0.037037,0.285714,0.0,0,0.548348,0.314964,0.75,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
161327,0.113208,0.090909,0.625000,0.026455,0.047619,0.0,0,0.587308,0.075554,0.50,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161328,0.245283,0.212121,0.333333,0.037037,0.047619,0.0,0,0.426275,0.125824,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
161329,0.660377,0.272727,0.104167,0.027513,0.095238,0.0,0,0.494398,0.017504,0.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# show normalized DataFrame columns info
credit_df_norm.describe()


,age,credit_month,tariff_id,monthly_income,credit_count,overdue_credit_count,open_account_flg,score_shk_real,credit_sum_real,marital_status_int,...,СЕВЕРОКАВКАЗСКИЙ_real,СЕВЕРОЗАПАДНЫЙ_real,OTHER_real,ПРИВОЛЖСКИЙ_real,ЮЖНЫЙ_real,СИБИРСКИЙ_real,УРАЛЬСКИЙ_real,ДАЛЬНЕВОСТОЧНЫЙ_real,gender_income,region_income
count,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,...,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000,161331.000000
mean,0.359307,0.242144,0.334086,0.037477,0.100359,0.015354,0.164922,0.476354,0.117596,0.544232,...,0.028370,0.141975,0.041548,0.212910,0.109031,0.092034,0.095574,0.057088,0.479765,0.274986
std,0.196233,0.107360,0.245450,0.026801,0.084637,0.070500,0.371111,0.125009,0.082084,0.183922,...,0.166029,0.349026,0.199555,0.409366,0.311678,0.289075,0.294007,0.232011,0.418772,0.348703
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.207547,0.212121,0.104167,0.021164,0.047619,0.000000,0.000000,0.386009,0.061299,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.320755,0.212121,0.333333,0.031746,0.095238,0.000000,0.000000,0.468757,0.092975,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000
75%,0.471698,0.272727,0.625000,0.047619,0.142857,0.000000,0.000000,0.559614,0.147173,0.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
# get predicted probabilities
prob_pred_gauss = gaussian_pipe.predict_proba(credit_df_test_norm[features])

# get predicted labels
y_pred_gauss = np.argmax(prob_pred_gauss, axis=1)

# get max class probabilities
y_prob_gauss = np.amax(prob_pred_gauss, axis=1)


In [26]:
# calculate metrics for GaussianNB
calculate_metrics('GaussianNB', y_prob_gauss, y_pred_gauss, credit_df_test_norm[target])


F1 score: 0.22183558068725534
Accuracy: 0.7782323044502293
AUC: 0.407270608735426


Используем логистическую регрессию (LogisticRegression). С помощью GridSearchCV найдем лучшую комбинацию из параметров для штрафа (penalty) и подборщика решений (solver). Т. к. согласно документации, не каждый параметр для штрафа подходит каждому подборщику решений, составим две сетки параметров для GridSearchCV и запустим его дважды. Результаты объединяем в DataFrame res_df.

In [27]:
params_grid = [{'penalty': ('l1', 'l2'), 'solver': ('liblinear', 'saga')},
               {'penalty': ('l2', None), 'solver': ('lbfgs', 'newton-cg', 'sag', 'saga')}]

grid_search = GridSearchCV(LogisticRegression(), params_grid, n_jobs=os.cpu_count())
grid_search = grid_search.fit(credit_df_train_norm[features], credit_df_train_norm[target])
res_df = pd.DataFrame(grid_search.cv_results_)


In [20]:
interested_columns = [col for col in res_df.columns if 'mean_' in col or 'param_' in col]
res_df = res_df[interested_columns].sort_values(by=['mean_test_score'], ascending=False)
res_df


,mean_fit_time,mean_score_time,param_penalty,param_solver,mean_test_score
0,122.251160,0.020549,l1,liblinear,0.834576
1,5.895035,0.016753,l1,saga,0.834576
2,2.503964,0.035921,l2,liblinear,0.834576
3,9.663939,0.017308,l2,saga,0.834576
5,3.027054,0.033048,l2,newton-cg,0.834576
6,7.509453,0.017082,l2,sag,0.834576
7,8.066569,0.021718,l2,saga,0.834576
8,1.823786,0.034574,None,lbfgs,0.834576
4,1.825101,0.025208,l2,lbfgs,0.834570
9,3.422466,0.032939,None,newton-cg,0.834570


По таблице выше видно, что точнее и быстрее логистическая регрессия работает с параметрами solver=liblinear, penalty=l1.

In [30]:
lr = grid_search.best_estimator_
lr


LogisticRegression(penalty='l1', solver='liblinear')

In [31]:
# get predicted probabilities
prob_pred_lr = lr.predict_proba(credit_df_test_norm[features])

# get predicted labels
y_pred_lr = np.argmax(prob_pred_lr, axis=1)

# get max class probabilities
y_prob_lr = np.amax(prob_pred_lr, axis=1)


In [32]:
# calculate metrics for LogisticRegression
calculate_metrics('LogisticRegression', y_prob_lr, y_pred_lr, credit_df_test_norm[target])


F1 score: 0.0
Accuracy: 0.8396553861410686
AUC: 0.3778520629970764


Линейная регрессия (LinearRegression) + PolynomialFeatures

Создадим новые свойства, используя PolynomialFeatures, из свойств 'credit_month', 'tariff_id', 'credit_sum_real'. Новые колонки объединим с текущей таблицей credit_df. Исключим три исходные колонки 'credit_month', 'tariff_id', 'credit_sum_real'. Разделим данные на обучающие и тестовые. Используем линейную регрессию в качестве классификатора.

In [33]:
features_names_in = ['credit_month', 'tariff_id', 'credit_sum_real']

# create pipe for transform features, normalize and classificate date
linear_pipe = Pipeline(steps=[('transformer', PolynomialFeatures(degree=2, interaction_only=False)),
                              ('model', LinearRegression(n_jobs=os.cpu_count()))])

# create new features from 'credit_month', 'tariff_id' and 'credit_sum_real' columns
features_array = linear_pipe['transformer'].fit_transform(credit_df[features_names_in], credit_df[target])[:, 1:]
all_polynom_names = linear_pipe['transformer'].get_feature_names_out()[1:]

# get columns only with new features 
new_features_names = list(set(all_polynom_names).difference(features_names_in))
print('New features:\n', new_features_names, '\n')

# get columns without old features
df_names = list(set(credit_df.columns).difference(features_names_in))

# create DataFrame with new columns and concat with credit_df
credit_df_poly = pd.DataFrame(data=features_array, columns=all_polynom_names)
credit_df_poly = pd.concat([credit_df_poly[new_features_names], credit_df[df_names]], axis=1)

extended_features = get_features(credit_df_poly, target)

# split dataset to train and test
poly_df_train, poly_df_test = train_test_split(credit_df_poly, test_size=0.1, random_state=0)

# fit linear regression model 
linear_pipe = linear_pipe.fit(poly_df_train[extended_features], poly_df_train[target])

# show transformed normalized DataFrame
credit_df_poly


New features:
 ['tariff_id^2', 'credit_month credit_sum_real', 'credit_sum_real^2', 'credit_month tariff_id', 'credit_month^2', 'tariff_id credit_sum_real'] 



,tariff_id^2,credit_month credit_sum_real,credit_sum_real^2,credit_month tariff_id,credit_month^2,tariff_id credit_sum_real,ДАЛЬНЕВОСТОЧНЫЙ_real,ЮЖНЫЙ_real,job_position_int,ЦЕНТРАЛЬНЫЙ_real,...,УРАЛЬСКИЙ_real,marital_status_int,credit_count,education_int,open_account_flg,СИБИРСКИЙ_real,age,monthly_income,region_income,СЕВЕРОКАВКАЗСКИЙ_real
0,0.010851,2.288405,0.052368,1.041667,100.0,0.023838,0.0,0.0,12,1.0,...,0.0,3,0.142857,1,0,0.0,0.150943,0.058201,1.0,0.0
1,0.010851,1.192919,0.014231,1.041667,100.0,0.012426,0.0,0.0,12,0.0,...,0.0,3,0.333333,1,0,0.0,0.264151,0.036508,0.0,1.0
2,0.000000,0.997638,0.005078,0.000000,196.0,0.000000,0.0,0.0,12,0.0,...,0.0,2,0.095238,3,0,0.0,0.150943,0.032804,0.0,0.0
3,0.390625,2.005079,0.040203,6.250000,100.0,0.125317,0.0,0.0,12,0.0,...,0.0,2,0.047619,3,0,0.0,0.358491,0.068783,0.6,0.0
4,0.010851,2.779575,0.077260,1.041667,100.0,0.028954,0.0,0.0,5,0.0,...,0.0,2,0.333333,1,0,0.0,0.471698,0.058201,0.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161326,0.010851,3.779564,0.099202,1.250000,144.0,0.032809,0.0,0.0,12,0.0,...,0.0,3,0.285714,1,0,0.0,0.169811,0.037037,0.0,0.0
161327,0.390625,0.453321,0.005708,3.750000,36.0,0.047221,0.0,0.0,12,0.0,...,0.0,2,0.047619,3,0,0.0,0.113208,0.026455,0.0,0.0
161328,0.111111,1.258236,0.015832,3.333333,100.0,0.041941,1.0,0.0,12,0.0,...,0.0,3,0.047619,1,0,0.0,0.245283,0.037037,0.0,0.0
161329,0.010851,0.210054,0.000306,1.250000,144.0,0.001823,0.0,0.0,8,0.0,...,0.0,1,0.095238,3,0,0.0,0.660377,0.027513,0.0,0.0


In [34]:
# get predicted probabilities
prob_pred_linear = linear_pipe.predict(poly_df_test[extended_features])

# get predicted labels
PROB_THRESHOLD = 0.26
y_pred_linear = (prob_pred_linear > PROB_THRESHOLD).astype(int)


In [35]:
# calculate metrics for LinearRegression
calculate_metrics('LinearRegression', prob_pred_linear, y_pred_linear, poly_df_test[target])


F1 score: 0.24312796208530807
Accuracy: 0.802032973844056
AUC: 0.6574400560298868


Классификатор RandomForestClassifier

In [ ]:
params_dict = {'n_estimators': (50, 100, 150),
               'max_depth': (20, 40, 60, None),
               'min_samples_split': (10, 20, 30, 40, 50),
               'min_samples_leaf': range(1, 6),
               'bootstrap': (True, False)}

grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy', max_features='log2'), 
                           params_dict, n_jobs=os.cpu_count(), verbose=2)
grid_search = grid_search.fit(credit_df_train_norm[features], credit_df_train_norm[target])
res_df = pd.DataFrame(grid_search.cv_results_)


In [17]:
interested_columns = [col for col in res_df.columns if 'mean_' in col or 'param_' in col]
res_df = res_df[interested_columns].sort_values(by=['mean_test_score'], ascending=False)
res_df


,mean_fit_time,mean_score_time,param_bootstrap,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,mean_test_score
307,21.546947,0.442464,False,20,1,30,100,0.835864
321,10.857057,0.228068,False,20,2,30,50,0.835857
349,20.955576,0.417052,False,20,4,20,100,0.835789
305,32.862564,0.770023,False,20,1,20,150,0.835782
19,16.012811,0.430171,True,20,2,20,100,0.835768
...,...,...,...,...,...,...,...,...
526,26.299373,0.710477,False,None,1,10,100,0.832400
376,27.897235,0.735213,False,40,1,10,100,0.832304
450,13.274399,0.308497,False,60,1,10,50,0.831980
375,13.855656,0.343523,False,40,1,10,50,0.831794


По таблице выше видно, что точнее классификатор работает с параметрами bootstrap=False, criterion=entropy, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=30, n_estimators=100.


In [23]:
forest = grid_search.best_estimator_
forest


RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=20,
                       max_features='log2', min_samples_split=30)

In [19]:
# get predicted probabilities
prob_pred_forest = forest.predict_proba(credit_df_test_norm[features])

# get predicted labels
y_pred_forest = np.argmax(prob_pred_forest, axis=1)

# get max class probabilities
y_prob_forest = np.amax(prob_pred_forest, axis=1)


In [20]:
# calculate metrics for RandomForestClassifier
calculate_metrics('RandomForestClassifier', y_prob_forest, y_pred_forest, credit_df_test_norm[target])


F1 score: 0.0565345080763583
Accuracy: 0.8407090616090244
AUC: 0.2539360181748801


In [36]:
analyze_df


,Model,F1_score,Accuracy,AUC
0,RandomForestClassifier,0.056535,0.840709,0.253936
1,GaussianNB,0.221836,0.778232,0.407271
2,LogisticRegression,0.000000,0.839655,0.377852
3,LinearRegression,0.243128,0.802033,0.657440


Как видно из результирующей таблицы, по точности 'accuracy' лучше всего работает RandomForestClassifier, но у линейной регрессии самые адекватые значения по всем метрикам одновременно. 